In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
next_layer = "silver"

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
# Dynamic select query to read the data for specific proc_date
dynamic_query = f""" CREATE OR REPLACE TEMP VIEW temp_pos_wcm_saleout AS SELECT *
FROM udp_wcm_pro.udp_wcm_bronze_pro.pos_wcm_saleout
WHERE ProcDate like '{str(proc_date).replace('-', '')}%'
"""

# Execute the query
spark.sql(dynamic_query)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_tranline
(
  hash_id BIGINT,
  receipt_no STRING,
  pos_no STRING,
  store_id STRING,
  calday DATE,
  caltime STRING,
  price Double,
  sale_qty Double,
  uom STRING,
  product_id STRING,
  barcode STRING,
  sale_amt Double,
  tran_no STRING,
  brand STRING,
  serial_no STRING,
  proc_date TIMESTAMP
) 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'true',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'true'
)
""")

In [0]:
spark.sql(
    f"""
CREATE OR REPLACE TEMP VIEW view_pos_wcm_saleout_tranline as
WITH dat AS (
    SELECT  
             LTRIM('0',main_tbl.ReceiptNo) AS receipt_no,
             LTRIM('0',main_tbl.PosNo) AS pos_no,
             LTRIM('0',main_tbl.StoreCode) AS store_id, 
             TO_DATE(main_tbl.CalendarDay, 'MM/dd/yyyy HH:mm:ss') AS calday,
             date_format(to_timestamp(substr(main_tbl.TranTime, 1, 6), 'HHmmss'),'HH:mm:ss') AS caltime,
             transline1.Price AS price, 
             transline1.Posquantity AS sale_qty,
             transline1.uom AS uom,
             transline1.article AS product_id,
             transline1.barcode AS barcode,
             transline1.amount AS sale_amt,
             transline1.TranNo AS tran_no,
             transline1.brand AS brand,
             transline1.SerialNo AS serial_no,
             transline1.MemberPointsEarn as member_points_earn,
             transline1.MemberPointsRedeem as member_points_redeem,
             TO_TIMESTAMP(main_tbl.ProcDate, 'yyyyMMddHHmmss') AS proc_date
        FROM temp_pos_wcm_saleout AS main_tbl
        LATERAL VIEW EXPLODE(main_tbl.TransLine) AS transline1
     ),
     RANKED AS (
         SELECT  
             farm_fingerprint(CONCAT(
                 COALESCE(CAST(receipt_no AS STRING), ''), 
                 COALESCE(CAST(tran_no AS STRING), ''), 
                 COALESCE(CAST(store_id AS STRING), '')
             )) AS hash_id,
             *,
             ROW_NUMBER() OVER (PARTITION BY receipt_no, tran_no, store_id ORDER BY proc_date DESC) AS rn
         FROM DAT
     )
     SELECT *  EXCEPT(rn) FROM RANKED WHERE rn = 1    
"""
)

In [0]:
spark.sql(f"DELETE FROM {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_tranline WHERE RECEIPT_NO IN (SELECT RECEIPT_NO FROM view_pos_wcm_saleout_tranline)")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_tranline 
(hash_id, receipt_no, pos_no, store_id, calday, caltime, price, sale_qty, uom, product_id, barcode, sale_amt, tran_no, brand, serial_no, proc_date, member_points_earn, member_points_redeem)
SELECT hash_id, receipt_no, pos_no, store_id, calday, caltime, price, sale_qty, uom, product_id, barcode, sale_amt, tran_no, brand, serial_no, proc_date, member_points_earn, member_points_redeem
FROM view_pos_wcm_saleout_tranline
"""
)